### Домашнее задание:

Задание:

1. Взять данные из
https://www.kaggle.com/datasets/mrapplexz/bashim-quotes
и обучить модель GPT для генерации своих цитат.

2. Взять новостные данные из
https://github.com/natasha/corus
load_lenta2. Нам понадобится сам текст и заголовок.
Обучить модель T5/ или GPT для генерации заголовков для статей.


In [ ]:
import pandas as pd
import json
import re
!pip install transformers
!pip install datasets
pd.set_option('max_colwidth', 400)
import re
from sklearn.model_selection import train_test_split
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
import gc
import torch

### Генерация цитат:

In [ ]:
# Скачаем датасет

!pip install -q kaggle
!mkdir ~/.kaggle
!cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mrapplexz/bashim-quotes
!kaggle datasets download -d mrapplexz/bashim-quotes
!unzip /content/bashim-quotes.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
bashim-quotes.zip: Skipping, found more recently modified local copy (use --force to force download)
bashim-quotes.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/bashim-quotes.zip
replace dataset.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
data = pd.read_json('/content/dataset.jsonl', lines=True)
data.head()

,id,date,rating,text
0,1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"
1,2,2004-08-30 11:25:00+00:00,25105.0,"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/"
2,3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаза?"" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!"
3,4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать в палату к девочкам?! - Если б мы были больные - мы б бегали к другим мальчикам"""
4,5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с субейзом под одбц. \n<Ohtori_Akio> лучше бы мы жили в пещере с гоблинами.


In [ ]:
data.shape

(81497, 4)

In [ ]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = texts.strip()
        summary = re.sub(r'<.*?>', '', summary)
        data += summary + ' '
    f.write(data)

In [ ]:
train, test = train_test_split(data, test_size=0.3)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [ ]:
print('Train dataset length: ' + str(len(train)))
print('Test dataset length: ' + str(len(test)))

Train dataset length: 57047
Test dataset length: 24450


In [ ]:
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

#sberbank-ai/rugpt3large_based_on_gpt2
#sberbank-ai/rugpt3medium_based_on_gpt2
#sberbank-ai/rugpt3small_based_on_gpt2

tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')
model = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
tokenizer.decode(train_dataset[2])

' тихо, тайно... :) весь город от грохота просыпался. :)  старшее поколение хотело поехать в Париж и умереть, нынешнее желает друг другу поехать в Бобруйск и выпить яду )  на самом деле я добрый, только об этом никто не знает\n *мрачно* а те кто знают -- уже никому не расскажут...  Макс сидит на работе и читает онигдоды :) \n Только не сидит, а полулежит; и не анекдоты, а форумы; и не читает, а просматривает :-) \n А так все именно так, как Свер сказал :-)  интересно а моя даза бан'

In [ ]:
training_args = TrainingArguments(
    output_dir='./gpt2-chief', 
    overwrite_output_dir=True,
    num_train_epochs=10, 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4, 
    eval_steps=100, # Number of update steps between two evaluations.
    save_steps=400, # after # steps model is saved
    warmup_steps=500, # number of warmup steps for learning rate scheduler
    optim = 'adamw_torch',
    report_to='all'
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

***** Running training *****
  Num examples = 127
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 160


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=160, training_loss=3.0104391098022463, metrics={'train_runtime': 89.6711, 'train_samples_per_second': 7.081, 'train_steps_per_second': 1.784, 'total_flos': 147431235256320.0, 'train_loss': 3.0104391098022463, 'epoch': 5.0})

In [ ]:
trainer.save_model()
tokenizer.save_pretrained('gpt_quotes')
model.save_pretrained('model_gpt_quotes')

Saving model checkpoint to ./gpt2-chief
Configuration saved in ./gpt2-chief/config.json
Model weights saved in ./gpt2-chief/pytorch_model.bin
tokenizer config file saved in gpt_quotes/tokenizer_config.json
Special tokens file saved in gpt_quotes/special_tokens_map.json
tokenizer config file saved in gpt_quotes/tokenizer_config.json
Special tokens file saved in gpt_quotes/special_tokens_map.json
Configuration saved in model_gpt_quotes/config.json
Model weights saved in model_gpt_quotes/pytorch_model.bin


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/gpt_quotes')
model_new = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/model_gpt_quotes').to('cuda')

In [ ]:
def generate_quotes(prefix):
    tokens = tokenizer(prefix, return_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]
    output = model_new.generate(
        **tokens, 
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        max_length=size + 50, 
        repetition_penalty=5., 
        # temperature=5,
        num_beams=80,
        # early_stopping=True,
        no_repeat_ngram_size=1)

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    print(f'{prefix} {result}\n *******\n')

In [ ]:
generate_quotes('Эээ')
generate_quotes('Ну что ж')
generate_quotes('Сегодня утром')
generate_quotes('Я хочу сказать, что')
generate_quotes('Кручу верчу')
generate_quotes('Не надо изысканности')
generate_quotes('О!')

Эээ ... а что это за зверь такой "зоопарк" и чем он отличается от обычного зоопарка?
 ЗОООПАРК (нем. Zoologischer Aufklärungsgemeinschaft) —
 *******

Ну что ж , приступим&hellip;
1. Запускаем GNU/Linux (или любой другой дистрибутив) с правами суперпользователя и пишем sudo apt-get install gn
 *******

Сегодня утром  проснулась от того, что кто-то трясет меня за плечо. Открыла глаза и обомлела - на мне была моя любимая футболка с надписью "Happy Birthday" (С ДНЁМ РОЖДЕНИЯ!!!)
 *******

Я хочу сказать, что  у нас есть все шансы стать чемпионами мира по футболу в 2018 году&raquo;, &mdash; сказал Мутко на встрече с президентом ФИФА Йозефом Блаттером.
Член исполкома Российского футбольного союза (РФС) Сергей
 *******

Кручу верчу  - не могу найти, кто же на самом деле сбил малайзийский "Боинг", летевший из Амстердама в Куала-Лумпур рейсом MH17" авиакомпании Malaysia Airlines 17 июля 2014 года.
UPD
 *******

Не надо изысканности &raquo;, &mdash; сказал мне как-то один мой хороший знакомый. 

### Генерация заголовков:

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2
!bzip2 -d lenta-ru-news.csv.bz2

In [3]:
data_lenta = pd.read_csv('/content/lenta-ru-news.csv')
data_lenta.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,"Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по ш...",Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова отложено,"Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени. \n«Русский инвалид», 16 сентября 1914 года.",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим расположением австрийский аэроплан, собиравшийся бросить бомбы, взлетел на воздух, атаковал неприятеля и протаранил неприятельский аппарат, предотвратив жертвы в наших войсках. Сам Нестеров при этом погиб смертью героя. По словам доставленных в Киев пленных австрийских офицеров, всей неприятельской арм...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,"Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки пробрались между фортов в самый город, офицеру пришло в голову доверить бульдогу письмо, в котором он посылал успокоительную весть своим родителям...",Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передают, что туда доставлен «швабский зверь» — взятый в плен прусский майор Прейскер, бывший комендант Калиша. Это — здоровый детина, с типично прусским наглым лицом и мутными глазами. Когда этого «зверя» вели с партией пленных по улице и из толпы раздались крики и проклятия, Прейскер трусливо замешался в толпу пленных, стараясь скрыться. Зато в заключен...",Библиотека,Первая мировая,1914/09/18


In [4]:
data_lenta.shape

(800975, 6)

In [5]:
data_lenta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     800975 non-null  object
 1   title   800975 non-null  object
 2   text    800970 non-null  object
 3   topic   738973 non-null  object
 4   tags    773756 non-null  object
 5   date    800975 non-null  object
dtypes: object(6)
memory usage: 36.7+ MB


In [6]:
data_lenta.dropna(subset=['text'], inplace=True)
data_lenta.reset_index(inplace=True)

In [7]:
train, test = train_test_split(data_lenta[:10000], test_size=0.3)

In [8]:
print('Train dataset length: ' + str(len(train)))
print('Test dataset length: ' + str(len(test)))

Train dataset length: 7000
Test dataset length: 3000


In [9]:
max_len_sum = int(np.mean([len(data_lenta['text'][txt]) for txt in range(len(data_lenta))])/2.5)
max_len_tl = int(np.mean([len(data_lenta['title'][txt]) for txt in range(len(data_lenta))]))

In [10]:
max_len_sum

532

In [11]:
max_len_tl

55

In [83]:
dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(test)

In [13]:
dataset_train

Dataset({
    features: ['index', 'url', 'title', 'text', 'topic', 'tags', 'date', '__index_level_0__'],
    num_rows: 7000
})

In [14]:
dataset_test

Dataset({
    features: ['index', 'url', 'title', 'text', 'topic', 'tags', 'date', '__index_level_0__'],
    num_rows: 3000
})

In [15]:
model_name = 'IlyaGusev/rut5_base_sum_gazeta'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [84]:
def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], batch['title'], padding='max_length', truncation=True, max_length=max_len_sum)
    # tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    # tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train = dataset_train.remove_columns(['index', 'url', 'title', 'text', 'topic', 'tags', 'date', '__index_level_0__'])
dataset_test = dataset_test.remove_columns(['index', 'url', 'title', 'text', 'topic', 'tags', 'date', '__index_level_0__'])

  0%|          | 0/875 [00:00<?, ?ba/s]

  0%|          | 0/375 [00:00<?, ?ba/s]

In [85]:
dataset_train

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 7000
})

In [18]:
dataset_train.save_to_disk('./')
dataset_test.save_to_disk('./')

In [19]:
model.num_parameters()

244309248

In [86]:
output_dir = 'lenta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.001,
    evaluation_strategy='steps', # Run evaluation every eval_steps,
    # save_strategy='epoch',
    save_steps=500, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    # run_name='run_gazeta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model='loss', # Use loss to evaluate best model.
    greater_is_better=False, # Best model is the one with the lowest loss, not highest.
    report_to='all',
    optim = 'adamw_torch',
    max_steps=500
)

PyTorch: setting up devices


In [87]:
gc.collect()

361

In [88]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
***** Running training *****
  Num examples = 7000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 500


Step,Training Loss,Validation Loss
500,0.033400,0.001130


***** Running Evaluation *****
  Num examples = 3000
  Batch size = 2
Saving model checkpoint to lenta/output/checkpoint-500
Configuration saved in lenta/output/checkpoint-500/config.json
Model weights saved in lenta/output/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from lenta/output/checkpoint-500 (score: 0.0011304415529593825).


TrainOutput(global_step=500, training_loss=0.03335163497924805, metrics={'train_runtime': 751.7238, 'train_samples_per_second': 1.33, 'train_steps_per_second': 0.665, 'total_flos': 706288988160000.0, 'train_loss': 0.03335163497924805, 'epoch': 0.14})

In [29]:
trainer.save_model(output_dir + '/model')

Saving model checkpoint to lenta/output/model
Configuration saved in lenta/output/model/config.json
Model weights saved in lenta/output/model/pytorch_model.bin


In [34]:
device = 'cuda'

In [89]:
def title_generate(text_index):

    input_text = test['text'][text_index]

    with torch.no_grad():
        tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

        source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
        source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

        generated_ids = model.generate(
            input_ids=source_ids,
            attention_mask=source_mask, 
            max_length=20,
            num_beams=10,
            # temperature = 1.3,
            repetition_penalty=1., 
            length_penalty=2, 
            early_stopping=True,
            no_repeat_ngram_size=1
        )

        pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        print(f"PREDICTION: {pred}")

In [91]:
INX = 7471
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)

TEXT: Испанская "Валенсия" сотворила очередную сенсацию нынешнего розыгрыша Лиги Чемпионов. Подопечные Эктора Купера обыграли по сумме двух матчей знаменитую "Барселону" и вышли в финал. Впрочем, уже после первой полуфинальной игры сомнений в конечном успехе "Валенсии" не было, ведь первый матч она выиграла (4:1). В ответном матче "Барселона" с первых минут начала осаждать ворота гостей. Однако реальных угроз каталонцы не создавали, хотя несколько раз голкиперу "Валенсии" приходилось вступать в игру. Первый тайм закончился нулевой ничьей. Вторая половина игры началась с атак футболистов "Валенсии", которые могли забить как минимум два мяча. "Барселона" ответила атакой, во время которой арбитр матча вполне мог назначить пенальти: в штрафной "Валенсии" явно сбили Ривалдо. Однако свисток судьи молчал, а через пару минут Мендьетта точным ударом лишил "Барсу" последних надежд. В оставшееся время Рональд де Бур и Филипп Коку забили в ворота "Валенсии" два мяча и принесли хозяевам победу (2:1

In [92]:
INX = 1869
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)

TEXT: Компания Amazon.com объявила в среду, что больше не будет продавать своим клиентам из Германии издание Mein Kampf, сообщает Washington Post. Это решение связано с жалобой немецкой антинацистской организации Simon Wiesenthal Center и немецких властей, направленной в Amazon.com некоторое время назад. Издание Mein Kampf на немецком языке в Германии запрещено. До сих пор немецкие покупатели могли заказать в Amazon.com только англоязычную версию книги, так как статус английского перевода книги в Германии остается не вполне ясным. Решение руководства крупнейшего онлайнового книжного магазина вообще прекратить продавать Mein Kampf немцам было встречено представителями правительственных и общественных организаций Германии с восторгом. Между тем, основной конкурент Amazon.com - книжный магазин Barnes & Noble Inc., также продающий Mein Kampf своим клиентам, в том числе из Германии, не собирается прекращать продажи, поскольку не считает их незаконными. Представители Simon Wiesenthal Center,

In [93]:
INX = 8139
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)

TEXT: Россия поставит Индии 40 вертолетов Ми-17-1В. Контракт на их поставку подписан в четверг в Дели между внешнеторговым объединением "Авиаэкспорт" и оборонным ведомством Индии, сообщает РИА "Новости. Общая сумма сделки составляет 170 миллионов долларов. Одновременно подписан контракт c российским объединением "Промэкспорт", которое доработает вертолеты для их использования в военно-транспортном варианте. В минобороне Индии отметили, что контракт на приобретение 40 вертолетов МИ-17 по своей значимости стоит сразу после подписанного в 1996 году масштабного соглашения о продаже Индии военных самолетов Су-30. Поставка вертолетов, выпускаемых Казанским вертолетным заводом, будет осуществляться партиями в течение 13 месяцев. К настоящему времени уже подготовлена первая партия из четырех машин. Они будут отправлены заказчику сразу после предоплаты индийской стороной контракта. МИ-17 являются модификацией МИ-8, вертолеты имеют потолок в пять километров и могут использоваться в условиях высо

In [94]:
INX = 6865
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)

TEXT: В субботу в Москве открылось заседание политсовета общероссийского политического общественного движения "Единство". Открыл заседание лидер движения "Единство", вице-премьер правительства РФ Сергей Шойгу, который заявил о необходимости реформирования возглавляемого им движения. На заседании было принято решение о преобразовании движения вполитическую партию и проведении учредительного съезда 27 мая, сообщает РИА "Новости". "За последние месяцы в политической жизни страны произошли серьезные изменения - Россия выбрала нового президента, новую Государственную думу, а по итогам прошедших выборов в стране появилась по-настоящему новая власть", - говорится в заявлении политсовета. В документе отмечается, что "России нужна партия будущего, партия общенародного единства, в противовес партиям прошлого. Такая партия должна стать одной из основ новой политической системы". Политсовет движения считает, что "Единство" может стать именно такой партией. C докладом о мероприятиях по подготовке к

In [95]:
INX = 1036
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)

TEXT: В среду в Берлине открылся общий комплекс посольств пяти северных стран. Как передает РИА "Новости", на торжестве присутствовали три королевские пары из Дании, Швеции и Норвегии, а также главы Финляндии и Исландии. Строительство облицованного медными листами комплекса длилось два года и обошлось в 100 миллионов марок.Теперь посольства имеют общее клубное помещение, столовую и зеленый дворик в историческом дипломатическом квартале Берлина, расположенном недалеко от Бранденбургских ворот. Со стороны Германии на открытии Северного дипломатического  комплекса присутствовали президент Йоханнес Рау и министр иностранных дел Йошка Фишер.
TITLE: В Берлине открылся Северный дипломатический комплекс
PREDICTION: В среду в Берлине открылся общий комплекс посольств пяти северных стран.
